In [1]:
import requests

In [2]:
parent_folder = '/Users/username/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/username/Desktop/ML/Recommendations/arcane/')

In [8]:
from sql.candidates.CandidateSQL import CandidateSQL

In [9]:
candidate_article_ids = CandidateSQL.get_all_candidate_article_ids()

In [10]:
len(candidate_article_ids)

11010

In [11]:
missing_summary_candidates = candidate_article_ids

In [12]:
import numpy as np

In [22]:
np.random.shuffle(missing_summary_candidates)

In [25]:
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

In [26]:
def create_podcast_attributes(article_id):
    r = requests.post('http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/get_article_attributes_from_llm', json={'articleId': article_id})
    if r.status_code == 200:
        return article_id

In [27]:
chunk_size = 50
num_chunks = (len(missing_summary_candidates) // chunk_size) + 1
for i in range(num_chunks):
    cur_article_ids = missing_summary_candidates[i * chunk_size: (i + 1) * chunk_size]
    start_time = time.time()
    completed = 0
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(create_podcast_attributes, art_id) for art_id in cur_article_ids]
        for future in as_completed(futures):
            if future.result():
                response = future.result()
                if response:
                    try:
                        completed += 1
                    except:
                        pass
    num_pending = len(missing_summary_candidates) - i * chunk_size
    print(f'Processed chunk {i} in {time.time() - start_time} seconds at {datetime.now()}. {num_pending} articles pending')

KeyboardInterrupt: 

In [28]:
i

0